In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, desc, concat, col, lit, unix_timestamp, row_number, rank
from pyspark.sql.functions import sum as fsum
from pyspark.sql.functions import count as fcount
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder\
    .config('spark.driver.extraClassPath'
            , '/home/user/shared/postgresql-42.3.1.jar')\
    .master('local')\
    .appName('homework_lesson_13')\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

In [4]:
pg_url = 'jdbc:postgresql://127.0.0.1/postgres'
pg_creds = {'user' : 'pguser', 'password' : 'secret'}

In [5]:
df_category = spark.read.jdbc(pg_url, table='category', properties=pg_creds)
df_film_category = spark.read.jdbc(pg_url, table='film_category', properties=pg_creds)
df_actor = spark.read.jdbc(pg_url, table='actor', properties=pg_creds)
df_film_actor = spark.read.jdbc(pg_url, table='film_actor', properties=pg_creds)
df_inventory = spark.read.jdbc(pg_url, table='inventory', properties=pg_creds)
df_rental = spark.read.jdbc(pg_url, table='rental', properties=pg_creds)
df_payment = spark.read.jdbc(pg_url, table='payment', properties=pg_creds)
df_film = spark.read.jdbc(pg_url, table='film', properties=pg_creds)
df_city = spark.read.jdbc(pg_url, table='city', properties=pg_creds)
df_address = spark.read.jdbc(pg_url, table='address', properties=pg_creds)
df_customer = spark.read.jdbc(pg_url, table='customer', properties=pg_creds)

In [6]:
#1. вывести количество фильмов в каждой категории, отсортировать по убыванию.

In [7]:
df1 = df_category.join(df_film_category
                 , df_film_category.category_id == df_category.category_id
                 , 'inner')\
    .groupby(df_category.name)\
    .count()\
    .withColumnRenamed("name", "category")\
    .withColumnRenamed("count", "films_qauntity")

In [8]:
df1.orderBy(desc('films_qauntity')).select(df1.category, df1.films_qauntity).show()

+-----------+--------------+
|   category|films_qauntity|
+-----------+--------------+
|     Sports|            74|
|    Foreign|            73|
|     Family|            69|
|Documentary|            68|
|  Animation|            66|
|     Action|            64|
|        New|            63|
|      Drama|            62|
|     Sci-Fi|            61|
|      Games|            61|
|   Children|            60|
|     Comedy|            58|
|     Travel|            57|
|   Classics|            57|
|     Horror|            56|
|      Music|            51|
+-----------+--------------+



In [9]:
#2. вывести 10 актеров, чьи фильмы большего всего арендовали, отсортировать по убыванию.

In [10]:
df_actor.select('actor_id', concat(col('first_name'), lit(' '), col('last_name')).alias('actor'))\
    .join(
    df_film_actor
    , df_film_actor.actor_id == df_actor.actor_id
    , 'inner'
    )\
    .join(
    df_inventory
    , df_inventory.film_id == df_film_actor.film_id
    , 'inner'
    )\
    .join(
    df_rental
    , df_rental.inventory_id == df_inventory.inventory_id
    , 'inner'
    )\
    .select('actor', 'rental_id')\
    .groupby('actor')\
    .count()\
    .orderBy(desc('count'))\
    .select('actor')\
    .show(10)

+------------------+
|             actor|
+------------------+
|       SUSAN DAVIS|
|    GINA DEGENERES|
|    MATTHEW CARREY|
|       MARY KEITEL|
|ANGELA WITHERSPOON|
|       WALTER TORN|
|       HENRY BERRY|
|       JAYNE NOLTE|
|        VAL BOLGER|
|     SANDRA KILMER|
+------------------+
only showing top 10 rows



In [11]:
#3. вывести категорию фильмов, на которую потратили больше всего денег.

In [12]:
df_category.join(df_film_category, df_film_category.category_id == df_category.category_id, 'inner')\
    .select(col('name').alias('category'), 'film_id')\
    .join(df_inventory, df_inventory.film_id == df_film_category.film_id, 'inner')\
    .join(df_rental, df_rental.inventory_id == df_inventory.inventory_id, 'inner')\
    .join(df_payment, df_payment.rental_id == df_rental.rental_id, 'inner')\
    .groupBy('category')\
    .sum('amount')\
    .orderBy(desc('sum(amount)'))\
    .select('category')\
    .show(1)

+--------+
|category|
+--------+
|  Sports|
+--------+
only showing top 1 row



In [13]:
#4. вывести названия фильмов, которых нет в inventory. Написать запрос без использования оператора IN.

In [14]:
df_film.join(df_inventory, df_inventory.film_id == df_film.film_id, 'leftanti')\
    .groupby('title')\
    .count()\
    .select('title')\
    .show()

+--------------------+
|               title|
+--------------------+
|       RAINBOW SHOCK|
|           GUMP DATE|
|         HOCUS FRIDA|
|    TREASURE COMMAND|
| CHINATOWN GLADIATOR|
|        WALLS ARTIST|
|      ARGONAUTS TOWN|
|       PSYCHO SHRUNK|
|   FIREHOUSE VIETNAM|
|DELIVERANCE MULHO...|
|       ROOF CHAMPION|
|        TADPOLE PARK|
|         APOLLO TEEN|
|       HATE HANDICAP|
|       PEARL DESTINY|
|COMMANDMENTS EXPRESS|
|        VOLUME HOUSE|
|     CROWDS TELEMARK|
|   RAIDERS ANTITRUST|
|    KILL BROTHERHOOD|
+--------------------+
only showing top 20 rows



In [15]:
#5. вывести топ 3 актеров, которые больше всего появлялись в фильмах в категории “Children”. 
#Если у нескольких актеров одинаковое кол-во фильмов, вывести всех.

In [16]:
qw = Window.orderBy(desc("count"))

In [17]:
df_actor.join(df_film_actor, df_film_actor.actor_id == df_actor.actor_id, 'inner')\
    .join(df_film_category, df_film_category.film_id == df_film_actor.film_id, 'inner')\
    .join(df_category, (df_category.category_id == df_film_category.category_id) & (df_category.name == 'Children'), 'inner')\
    .withColumn('actor', concat(col('first_name'), lit(' '), col('last_name')))\
    .groupby('actor')\
    .count()\
    .sort(desc('count'))\
    .withColumn("position", rank().over(qw))\
    .where(col('position') <= 3)\
    .select('actor')\
    .show()

21/11/08 19:25:49 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+
|        actor|
+-------------+
| HELEN VOIGHT|
|  SUSAN DAVIS|
|   MARY TANDY|
|   RALPH CRUZ|
|  WHOOPI HURT|
|KEVIN GARLAND|
+-------------+



In [18]:
#6. вывести города с количеством активных и неактивных клиентов (активный — customer.active = 1). 
#Отсортировать по количеству неактивных клиентов по убыванию.

In [19]:
df_city.join(df_address, df_address.city_id == df_city.city_id, 'inner')\
    .join(df_customer, df_customer.address_id ==df_address.address_id, 'inner')\
    .groupBy('city')\
    .agg(fsum('active').alias('active_customer_quantity'), fcount('active').alias('all_customer_quantity'))\
    .select('city', 'active_customer_quantity', (col('all_customer_quantity') - col('active_customer_quantity')).alias('inactive_customer_quantity'))\
    .sort(desc('inactive_customer_quantity'))\
    .show()

+----------------+------------------------+--------------------------+
|            city|active_customer_quantity|inactive_customer_quantity|
+----------------+------------------------+--------------------------+
|          Ktahya|                       0|                         1|
|Charlotte Amalie|                       0|                         1|
|         Wroclaw|                       0|                         1|
|       Pingxiang|                       0|                         1|
|     Szkesfehrvr|                       0|                         1|
|          Daxian|                       0|                         1|
|   Coatzacoalcos|                       0|                         1|
|         Bat Yam|                       0|                         1|
| Southend-on-Sea|                       0|                         1|
|        Uluberia|                       0|                         1|
|       Najafabad|                       0|                         1|
|     

In [20]:
#7. вывести категорию фильмов, у которой самое большое кол-во часов суммарной аренды в городах 
#(customer.address_id в этом city), и которые начинаются на букву “a”. 
#То же самое сделать для городов в которых есть символ “-”. Написать все в одном запросе.

In [21]:
w = Window.partitionBy("city").orderBy(desc("sum(rent_time)"))

In [22]:
df_city.filter(col("city").like("a%") | col("city").like("A%") | col("city").like("%-%"))\
    .join(df_address, df_address.city_id == df_city.city_id, 'inner')\
    .join(df_customer, df_customer.address_id == df_address.address_id, 'inner')\
    .join(df_rental, df_rental.customer_id == df_customer.customer_id, 'inner')\
    .where(df_rental.rental_date.isNotNull() & df_rental.return_date.isNotNull())\
    .join(df_inventory, df_inventory.inventory_id == df_rental.inventory_id, 'inner')\
    .join(df_film_category, df_film_category.film_id == df_inventory.film_id, 'inner')\
    .join(df_category, df_category.category_id == df_film_category.category_id , 'inner')\
    .withColumn("rent_time", (unix_timestamp('return_date') - unix_timestamp("rental_date")))\
    .select('city', df_category.name.alias('category'), 'rent_time')\
    .groupby('city', 'category')\
    .agg(fsum(col('rent_time')))\
    .withColumn("row_number", row_number().over(w))\
    .where(col('row_number') == 1)\
    .select('city', 'category')\
    .sort('city')\
    .show()

+--------------------+--------+
|                city|category|
+--------------------+--------+
|  A Corua (La Corua)|  Comedy|
|                Abha|  Sci-Fi|
|           Abu Dhabi|  Sci-Fi|
|                Acua|   Drama|
|               Adana|  Comedy|
|         Addis Abeba|  Family|
|                Aden|     New|
|               Adoni|Children|
|          Ahmadnagar|Children|
|            Akishima|Children|
|               Akron|  Sports|
|         Alessandria|  Comedy|
|Allappuzha (Allep...|     New|
|             Allende|  Travel|
|     Almirante Brown|  Sports|
|            Alvorada|  Sci-Fi|
|            Ambattur|   Games|
|          Amersfoort|  Sports|
|              Amroha|   Music|
|      Angra dos Reis|  Family|
+--------------------+--------+
only showing top 20 rows

